# Using ClassCOMPAS

This notebook serves as a tutorial for using ClassCOMPAS to post-process COMPAS output.

In [3]:
import os
import sys
pathNoteBook     = os.getcwd()
pathClassCOMPAS  = pathNoteBook + '/PythonScripts/'
sys.path.append(pathClassCOMPAS)

import numpy as np
import ClassCOMPAS

## Create instance of ClassCOMPAS
We first set class attributes. For details, see ClassCOMPAS.py

In [4]:
path = 'COMPAS_output/'                  # Path to directory containing COMPAS h5 file
fileName = 'COMPAS_CHE_30xZ_3000000.h5'  # Name of COMPAS HDF5 file
lazyData = True                          # True to store some additional quantities like mass ratios and chirp masses

We then set attributes related to sampling the initial mass function (IMF). Mlower, Mupper, and binaryFraction are needed to calculate the total star-forming mass as opposed to just the amount of stellar mass evolved by COMPAS.

Mlower and Mupper should correspond to the option specified in the pythonSubmit.py.

In [5]:
Mlower = 5.             # Lower mass cutoff in COMPAS's sampling from the Kroupa IMF 
Mupper = 150.           # Upper mass cutoff in COMPAS's sampling from the Kroupa IMF 
binaryFraction = 0.7    # Assummed fraction of stellar systems in binaries

# Now create instance of ClassCOMPAS
COMPASData = ClassCOMPAS.COMPASData(
    path=path,
    fileName=fileName,
    lazyData=lazyData,
    Mlower=Mlower,
    Mupper=Mupper,
    binaryFraction=binaryFraction
    )

ClassCOMPAS: Remember to self.setZgridAndCalcMassFormed()
                   then  self.setCOMPASDCOmask()
                   then  self.setCOMPASData()


## Calculate total star-forming mass
We then need to recover the total star-forming mass corresponding to our COMPAS simulation from (i) the total stellar mass drawn in the COMPAS run and (ii) the IMF used. This can be done simply by calling the setZgridAndCalcMassFormed method. This method creates the following class attributes:

(i) metallicityGrid: Array of the metallicities evolved in your COMPAS run

(ii) massFormedAtEachZ: Total star-forming mass at each metallicity

In [6]:
COMPASData.setZgridAndCalcMassFormed()

You can see the metallicities in your COMPAS run:

In [7]:
print(COMPASData.metallicityGrid)

[0.0001     0.00011885 0.00014125 0.00016788 0.00019953 0.00023714
 0.00028184 0.00033497 0.00039811 0.00047315 0.00056234 0.00066834
 0.00079433 0.00094406 0.00112202 0.00133352 0.00158489 0.00188365
 0.00223872 0.00266073 0.00316228 0.00375837 0.00446684 0.00530884
 0.00630957 0.00749894 0.00891251 0.01059254 0.01258925 0.01496236]


You can also see the corresponding amount of mass formed at each metallicity:

In [10]:
print(COMPASData.massFormedAtEachZ) # In Msol

[10914062.62547771 10869826.17883275 10880310.68209276 10842245.11900893
 10880968.85949892 10906846.74058563 10914669.55562297 10943169.69720117
 10918236.02016245 10867826.10407558 10874059.66831391 10850164.39689123
 10882938.83581023 10855643.89796973 10966923.40370443 10824912.72872888
 10933947.66061918 10945262.74546761 10912602.09039398 10920072.52719218
 10910833.7752537  10927451.69415953 10923768.73093803 10842602.55553873
 10887305.27589716 10946318.10072558 10960071.46727974 10890855.54839228
 10940849.86555227 10827570.71352417]


## Masking interesting systems
We now need to apply filters to extract the double compact objects (DCOs) of interest. This is accomplished by using the setCOMPASDCOmask method. This method allows you to filter

* DCOs of a particular type
* DCOs that merge within a Hubble time
* DCOs that are formed assumming the pessimistic option for common-envelopes (Hertzsprung-gap donors are not allowed to survive common-envelope events)

The setCOMPASDCOmask method creates the attribute DCOmask, which is a boolean array containing 'True' for systems of interest and 'False' otherwise corresponding to the systems in DoubleCompactObjects.

We set our filter below:

In [6]:
desiredDCO = 'BBH'        # Can take value 'BBH', 'BNS', 'BHNS', or 'all'
withinHubbleTime = True   # Mask for systems that merge within a Hubble time
pessimistic = True        # Assume pessimistic common-envelope prescription
noRLOFafterCEE = True     # Assume systems merge if there is immediate RLOF after a common-envelope episode

COMPASData.setCOMPASDCOmask(
    types=desiredDCO,
    pessimistic=pessimistic,
    withinHubbleTime=withinHubbleTime,
    noRLOFafterCEE=noRLOFafterCEE
    )

You can then print the number of systems of interest:

In [8]:
np.sum(COMPASData.DCOmask) # Print number of systems of interest

10646

## Setting the COMPAS data
Having created our mask for interesting systems, it only remains to run the setCOMPASData method carry out the masking of mask our data. This procedure creates the following masked attributes:

* seedsDCO: Seed
* metallicitySystems: Metallicity of each system
* delayTimes: Delay time (Myr)
* mass1
* mass2

The following attributes are only set if lazydata = True:
* q: A symmetrised mass ratio defined as the ratio of the smaller to larger mass object 
* mChirp: Chirp mass
* Hubble: Boolean array that is true if system of interest merges within a Hubble time

In [9]:
COMPASData.setCOMPASData()

In [10]:
# These arrays now contain properties only of the binaries of interest
print(COMPASData.metallicitySystems)
print(COMPASData.mChirp)
print(COMPASData.delayTimes)
print(COMPASData.mass1)

[0.0001     0.0001     0.0001     ... 0.01059254 0.01059254 0.01059254]
[10.74323879 28.98567745 32.30563026 ...  4.93805596  5.16415866
  5.46817157]
[  76.74304208   94.63040768   93.938463   ...    8.7038262  6428.33321746
   49.3884491 ]
[15.47559  33.2958   36.5311   ... 13.44527  12.94106   8.209575]


Note that if you wish to use the COMPASData class without actually having a COMPAS HDF5 output, you can create an instance of ClassCOMPAS by setting path=None, then manually set each array:
* MockData.metallicityGrid
* MockData.totalMassEvolvedPerZ  
* MockData.metallicitySystems
* MockData.delayTimes 
* MockData.mass1       
* MockData.mass2  

In [11]:
MockData = ClassCOMPAS.COMPASData(path=None)

path not set in instance of ClassCOMPAS
ClassCOMPAS: Remember to self.setZgridAndCalcMassFormed()
                   then  self.setCOMPASDCOmask()
                   then  self.setCOMPASData()


## Selection effects

In [17]:
##################################################################################################################
# The selection effects class assigns a probability of detecting a compact binary coalescence by integrating its
# SNR for given distance, position, inclination, and binary properties. It does so by Monte Carlo sampling of 
# uniformly distributed source-detector inclination, and defines the detection probability as the fraction of 
# Monte Carlo  extrinsic parameter realisations for which the observed SNR passes our threshold (Barrett et al., 
# 2018).
##################################################################################################################
import selection_effects

# Specify intrinsic parameters:
M1 = 40 # In Msol
M2 = 40 # In Msol
redshift = 0.1
distance = 463.4 # In Mpc

SNRthreshold = 8

##################################################################################################################
# There are two options for choosing a sensitivity curve:
# (i) 'O1'
# (ii) 'Design'
##################################################################################################################

sensitivity = 'O1' 

Pdet = selection_effects.detection_probability(M1, M2, redshift, distance, SNRthreshold, sensitivity=sensitivity)

In [18]:
print(Pdet)

0.776501


In [21]:
import h5py as h5
Data = h5.File('../../runs/COMPAS_CHE_30xZ_3000000.h5')
ZAMS_m1 = Data["SystemParameters"]['Mass@ZAMS_1'][()]
ZAMS_m2 = Data["SystemParameters"]['Mass@ZAMS_2'][()]

sortedidx = np.argsort(ZAMS_m1)
sortedZAMS_m1 = ZAMS_m1[sortedidx]
sortedZAMS_m2 = ZAMS_m2[sortedidx]
sorted_diff_ZAMSmass = sortedZAMS_m1-sortedZAMS_m2